In [11]:
import pertpy as pt
import scanpy as sc
import numpy as np
import pandas as pd
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

out_dir = 'data'

#### Download and process data

In [2]:
adata = pt.dt.norman_2019()
adata.X = adata.layers['counts']
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata, n_top_genes=2000)

#### Select 1-gene perturbatuons

In [3]:
single_gene_mask = np.array(['+' in p for p in adata.obs['perturbation_name'].values])
adata_1gene = adata[~single_gene_mask]

#### Compute DE genes

In [4]:
method = 't-test'
sc.tl.rank_genes_groups(
    adata_1gene,
    groupby='perturbation_name',
    reference='control',
    method=method,
    key_added=method,
    rankby_abs=True,
    n_genes=None,
    use_raw=False
);

/home/vinas/miniconda3/envs/scpert/lib/python3.12/site-packages/scanpy/tools/_rank_genes_groups.py:640: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns[key_added] = {}


In [ ]:
# Get DE analysis results as follows
df = sc.get.rank_genes_groups_df(adata_1gene, group=None, key=method)

#### Store data

In [12]:
adata_1gene.write(f'{out_dir}/1gene-norman.h5ad')